crf测试pku中文语料第2版

改进： 测试集从7/3划分改成从训练集中随机抽取一句带地名/人名的话。

结果： 预测的label与真实label仍一摸一样，即precision/recall仍全是1。

bug:  在做特征提取时，将label作为一个特征来训练了。

In [116]:
import random
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import pycrfsuite

In [117]:
if __name__=='__main__':
    train_sents = []
    f = open('./train_utf16.ner', 'r', encoding='utf_16_le')
    pairList = []
    for line in f:
        line = line.encode('utf-8').decode('utf-8-sig').strip()  # 去掉首字前的\ufeff
        if line:
            l = line.split(' ')
            pairList.append([l[0], l[1]])
        else:
            train_sents.append(pairList)
            pairList = []
    f.close()

In [118]:
print(len(train_sents))

23182


In [119]:
print(train_sents[888])

[['这', 'N'], ['里', 'N'], ['刊', 'N'], ['登', 'N'], ['湖', 'B-LOC'], ['南', 'I-LOC'], ['读', 'N'], ['者', 'N'], ['周', 'B-PER'], ['云', 'I-PER'], ['武', 'I-PER'], ['的', 'N'], ['来', 'N'], ['稿', 'N'], ['—', 'N'], ['—', 'N'], ['—', 'N']]


In [120]:
print(train_sents[889])

[['一', 'N'], ['周', 'N'], ['来', 'N'], ['，', 'N'], ['河', 'B-LOC'], ['南', 'I-LOC'], ['西', 'B-LOC'], ['峡', 'I-LOC'], ['符', 'B-PER'], ['祖', 'I-PER'], ['国', 'I-PER'], ['、', 'N'], ['江', 'B-LOC'], ['西', 'I-LOC'], ['广', 'B-LOC'], ['昌', 'I-LOC'], ['吴', 'B-PER'], ['小', 'I-PER'], ['毛', 'I-PER'], ['、', 'N'], ['河', 'B-LOC'], ['北', 'I-LOC'], ['石', 'B-LOC'], ['家', 'I-LOC'], ['庄', 'I-LOC'], ['马', 'B-PER'], ['强', 'I-PER'], ['等', 'N'], ['百', 'N'], ['余', 'N'], ['位', 'N'], ['朋', 'N'], ['友', 'N'], ['给', 'N'], ['本', 'N'], ['刊', 'N'], ['来', 'N'], ['信', 'N'], ['来', 'N'], ['稿', 'N'], ['。', 'N']]


In [121]:
test_sents = train_sents[888]

In [122]:
print(test_sents)

[['这', 'N'], ['里', 'N'], ['刊', 'N'], ['登', 'N'], ['湖', 'B-LOC'], ['南', 'I-LOC'], ['读', 'N'], ['者', 'N'], ['周', 'B-PER'], ['云', 'I-PER'], ['武', 'I-PER'], ['的', 'N'], ['来', 'N'], ['稿', 'N'], ['—', 'N'], ['—', 'N'], ['—', 'N']]


In [123]:
def word2features(sent, i):  # for i in range(len(sent))
    word = sent[i][0]
    label = sent[i][1]
    features = [  # 每个当前字的features
        'bias',
        'word=' + word,
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'label=' + label,
    ]
    if i > 0:
        word1 = sent[i - 1][0]
        label1 = sent[i - 1][1]
        features.extend([  # 前一个单词的features
            '-1:word=' + word1,
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:postag=' + label1,
        ])
    else:
        features.append('BOS')
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        label1 = sent[i + 1][1]
        features.extend([  # 后一个单词的features
            '+1:word=' + word1,
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:label=' + label1,
        ])
    else:
        features.append('EOS')
    return features

In [124]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, label in sent]


def sent2tokens(sent):
    return [token for token, label in sent]

In [125]:
print(sent2features(train_sents[0])[0])

['bias', 'word=藏', 'word.istitle=False', 'word.isdigit=False', 'label=N', 'BOS', '+1:word=书', '+1:word.istitle=False', '+1:label=N']


In [126]:
print(sent2labels(train_sents[0])[0])

N


In [127]:
X_train = [sent2features(s) for s in train_sents]  # 不加[]是generator类型，不会输出。加了[]，可直接输出加了[]之后的结果。
y_train = [sent2labels(s) for s in train_sents]

In [128]:
X_test = [sent2features(test_sents)]  # 格式 [[[每个word的特征],[每个word的特征]],（到此处是一个s）,[(每个word的特征)],...]，
y_test = [sent2labels(test_sents)] 

In [129]:
# verbose=False，是否在训练时输出debug信息
trainer = pycrfsuite.Trainer(verbose=False) 

In [130]:
# trainer中的数据
for xseq, yseq in zip(X_train, y_train): 
    trainer.append(xseq, yseq)  

In [131]:
# trainer的设置参数
trainer.set_params({
    'c1':1.0,
    'c2':1e-3,
    'max_iterations':50,
    'feature.possible_transitions':True
})

In [132]:
print('trainer.params():', trainer.params())

trainer.params(): ['feature.minfreq', 'feature.possible_states', 'feature.possible_transitions', 'c1', 'c2', 'max_iterations', 'num_memories', 'epsilon', 'period', 'delta', 'linesearch', 'max_linesearch']


In [133]:
# trainer得出的模型
trainer.train('conll2002_2.crfsuite')

In [134]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002_2.crfsuite')

In [135]:
print('test_sents:', test_sents)

test_sents: [['这', 'N'], ['里', 'N'], ['刊', 'N'], ['登', 'N'], ['湖', 'B-LOC'], ['南', 'I-LOC'], ['读', 'N'], ['者', 'N'], ['周', 'B-PER'], ['云', 'I-PER'], ['武', 'I-PER'], ['的', 'N'], ['来', 'N'], ['稿', 'N'], ['—', 'N'], ['—', 'N'], ['—', 'N']]


In [136]:
print(sent2tokens(test_sents))

['这', '里', '刊', '登', '湖', '南', '读', '者', '周', '云', '武', '的', '来', '稿', '—', '—', '—']


In [137]:
print('ture:', ' '.join(sent2labels(test_sents)))
print('predicted:', ' '.join(tagger.tag(sent2features(test_sents))))

ture: N N N N B-LOC I-LOC N N B-PER I-PER I-PER N N N N N N
predicted: N N N N B-LOC I-LOC N N B-PER I-PER I-PER N N N N N N


In [138]:
a = [['河','B-LOC'],['南','I-LO'],['人','N'],['在','N'],['天','B-LOC'],['安','I-LOC'],['门','I-LOC']]

In [139]:
print(sent2features(a)[0])

['bias', 'word=河', 'word.istitle=False', 'word.isdigit=False', 'label=B-LOC', 'BOS', '+1:word=南', '+1:word.istitle=False', '+1:label=I-LO']


In [140]:
print('ture:', ' '.join(sent2labels(a)))
print('predicted:', ' '.join(tagger.tag(sent2features(a))))

ture: B-LOC I-LO N N B-LOC I-LOC I-LOC
predicted: B-LOC I-LOC N N B-LOC I-LOC I-LOC


In [141]:
print('predicted:', ' '.join(tagger.tag(a)))

predicted: B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG


In [142]:
b = '湖南读者周云武的来稿'

In [143]:
print('predicted:', ' '.join(tagger.tag(['这', '里', '刊', '登', '湖', '南', '读', '者', '周', '云', '武', '的', '来', '稿', '—', '—', '—']
)))

predicted: B-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG I-ORG
